In [1]:
from dataset import Image_Dataset
import open_clip
from torch.utils.data import DataLoader
import torch
from tqdm import tqdm
torch.set_grad_enabled(False)
N = 10_000

In [2]:
pretraining_datasets = dict(open_clip.list_pretrained())

clip = open_clip.create_model("ViT-H-14", pretrained=pretraining_datasets["ViT-H-14"]).cuda()

for path in [
  "../../datasets/dog/",
  "../../datasets/bedroom/",
  "../../datasets/few-shot-images/AnimalFace-dog/img/",
  "../../datasets/few-shot-images/AnimalFace-cat/img/",
  "../../datasets/few-shot-images/fauvism-still-life/img/",
  "../../datasets/few-shot-images/moongate/img/",  
]:


    # dataset = Image_Dataset(root_dir="../../datasets/bedroom/", filetype="webp", resolution=(224,224))
    dataset = Image_Dataset(root_dir=path, filetype="jpg" if not "bedroom" in path else "webp", resolution=(224,224))

    dataloader = DataLoader(dataset, batch_size=8)

    min_clip_score = torch.tensor(1)
    i = 0
    cvs = []
    for x in tqdm(dataloader, total=N//8):
        if i > N:
            break
        i += 8
        img = x["image"].to("cuda")
        cv = clip.encode_image(img)
        cv = cv / cv.norm(dim=-1, keepdim=True)
        cvs.append(cv)
    cvs = torch.cat(cvs)
    min_clip_score = (cvs @ cvs.T).min()
    print(path, min_clip_score.item(), len(cvs))
    del dataloader
    del dataset
    del cv

1251it [09:07,  2.29it/s]                                                                                                                   


../../datasets/dog/ 0.12816427648067474 10008


1251it [09:06,  2.29it/s]                                                                                                                   


../../datasets/bedroom/ 0.010223356075584888 10008


  4%|███▉                                                                                                 | 49/1250 [00:21<08:34,  2.33it/s]


...

../../datasets/few-shot-images/AnimalFace-cat/img/ 0.4686216711997986 160


  1%|█▎                                                                                                   | 16/1250 [00:06<08:28,  2.43it/s]


../../datasets/few-shot-images/fauvism-still-life/img/ 0.3153287172317505 124


  1%|█▎                                                                                                   | 16/1250 [00:06<08:41,  2.37it/s]


../../datasets/few-shot-images/moongate/img/ 0.28092846274375916 128
